In [ ]:
train_dir = '/content/drive/MyDrive/data_vit/split_dataset/train'
test_dir = '/content/drive/MyDrive/data_vit/split_dataset/test'

In [ ]:
# !pip install torchinfo
# !pip install captum
# !pip install lime
# !pip install shap
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchinfo import summary
import matplotlib.pyplot as plt
from captum.attr import LayerGradCam, IntegratedGradients, visualization as viz
from lime import lime_image
import shap
from PIL import Image
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define your data preprocessing function
def preprocess_data(train_dir, test_dir, batch_size, num_workers=4):
    # Define transforms for data preprocessing
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Create datasets
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Split the training data into training and validation sets
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # Create data loaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, val_loader, test_loader

# Define your ViT model
class ViT(nn.Module):
    def __init__(self, num_classes):
        super(ViT, self).__init__()
        self.model = torchvision.models.vision_transformer.vit_base_patch16_224_in21k(pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Define the function to train the model
def train_model(model, train_loader, val_loader, optimizer, loss_fn, epochs, device):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                val_loss += loss.item() * images.size(0)
            val_loss /= len(val_loader.dataset)
            val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    return train_losses, val_losses

# Define the function to evaluate the model
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Define the function to generate explanations using Grad-CAM
def generate_grad_cam_explanations(model, image, target_class, device):
    model.eval()
    layer_grad_cam = LayerGradCam(model.model, model.model.encoder.blocks[-1].norm1)
    attributions_grad_cam = layer_grad_cam.attribute(image, target=target_class)
    vis = viz.visualize_image_attr_multiple(np.transpose(attributions_grad_cam.squeeze().cpu().detach().numpy(), (1,2,0)),
                                             np.transpose(image.squeeze().cpu().detach().numpy(), (1,2,0)),
                                             ["original_image", "heat_map"],
                                             ["all", "positive"],
                                             cmap="viridis",
                                             show_colorbar=True)
    plt.imshow(vis)
    plt.show()

# Define the function to generate explanations using Integrated Gradients
def generate_integrated_gradients_explanations(model, image, target_class, device):
    model.eval()
    integrated_gradients = IntegratedGradients(model.model)
    attributions_ig, delta = integrated_gradients.attribute(image, target=target_class, return_convergence_delta=True)
    vis_ig = viz.visualize_image_attr_multiple(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                                               np.transpose(image.squeeze().cpu().detach().numpy(), (1,2,0)),
                                               ["original_image", "heat_map"],
                                               ["all", "positive"],
                                               cmap="viridis",
                                               show_colorbar=True)
    plt.imshow(vis_ig)
    plt.show()

# Define the function to generate explanations using LIME
def generate_lime_explanations(model, image_path):
    model.eval()
    explainer = lime_image.LimeImageExplainer()
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    lime_explanation = explainer.explain_instance(image[0].cpu().permute(1, 2, 0).numpy(),
                                                   model,
                                                   top_labels=5,
                                                   hide_color=0,
                                                   num_samples=1000)
    explanation, mask = lime_explanation.get_image_and_mask(lime_explanation.top_labels[0],
                                                             positive_only=True,
                                                             num_features=5,
                                                             hide_rest=False)
    plt.imshow(explanation)
    plt.show()

# Define the function to generate explanations using SHAP
def generate_shap_explanations(model, image_path):
    model.eval()
    shap_explainer = shap.Explainer(model, image, output_rank_order='max')
    shap_values = shap_explainer.shap_values(image)
    shap.image_plot(shap_values, -image.numpy(), labels=class_names)

# Define paths to your train and test directories

train_dir ='/content/drive/MyDrive/data_vit/split_dataset/train'
test_dir = '/content/drive/MyDrive/data_vit/split_dataset/test'

# Define hyperparameters
batch_size = 32
num_epochs = 10
learning_rate = 1e-3

# Preprocess the data
train_loader, val_loader, test_loader = preprocess_data(train_dir, test_dir, batch_size)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the ViT model
num_classes = len(train_loader.dataset.classes)
model = ViT(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
train_losses, val_losses = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device)

# Evaluate the model
evaluate_model(model, test_loader, device)

# Generate explanations using Grad-CAM
image_path = 'example_image.jpg'  # Provide the path to your example image
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)
generate_grad_cam_explanations(model, image, target_class=0, device=device)

# Generate explanations using Integrated Gradients
generate_integrated_gradients_explanations(model, image, target_class=0, device=device)

# Generate explanations using LIME
generate_lime_explanations(model, image_path)

# Generate explanations using SHAP
generate_shap_explanations(model, image_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data_vit/split_dataset/train'